In [5]:
def format():
    data=open("1cumle").read().rsplit()
    output=open("1cum.txt","w")
    taglist=["[PER","[LOC","[ORG"]
    up=0
    tag=""
    a=0
    while a< len(data):
        a=up
        token=data[a]
        ne=""
        if token in taglist:
            tag=token[1:]
            a+=1
            token=data[a]
            output.write("%s B%s\n"%(token,tag))
            a+=1
            token=data[a]
            while token!="]":
                ne=ne+token+" "
                output.write("%s I%s\n"%(token,tag))
                a+=1
                token=data[a]
            a+=1
            up=a
        else : 
            output.write("%s O\n"%token)
            a+=1
            up=a



In [6]:

##formatting the NER annotated data into MP-MD format similar as in the examples
def format2():
    datal=open("reyyan.test.txt").readlines()
    output=open("hasimformat.test.txt","w")
    taglist=["[PER","[LOC","[ORG"]
    for datax in datal:
        data=datax.split()
        up=0
        tag=""
        a=0
        while a< len(data):
            a=up
            token=data[a]
            ne=""
            if token in taglist:
                tag=token[1:]
                a+=1
                token=data[a]
                output.write("%s " %token)
                a+=1
                token=data[a]
                while token!="]":
                    ne=ne+token+" "
                    output.write("%s " %token)
                    a+=1
                    token=data[a]
                a+=1
                up=a
            else : 
                output.write("%s "%token)
                a+=1
                up=a
        output.write("\n")

format2()

In [13]:
morptrain="afterMPMD.reyyan.train.txt"
morptest="afterMPMD.reyyan.test.txt"
data=open("afterMPMD.reyyan.train.txt").readlines()
datawostags=[]
tags=["<S>","</S>"]
print(data[0])
for line in data:
    if line.rsplit()[0] not in tags:
        datawostags.append(line)
print(datawostags[5])

<S> <S>+BSTag

ve ve[Conj]



In [8]:
##with datawostags every ith line corresponds to the ith words morpological analysis
### for now I will take the latest POS tag after inflections if there are any
##findPOS finds the POS tag given the line of analysis 
## if mod is 1 returns last POS
## if mod is 0 returns root POS
s=datawostags[11].rsplit()[1]
print(s+" %d "%s.find("[Noun"))

def findPOS(morphanalysis,mod):
    Poslist=["[Verb","[Adj","[Adv","[Noun","[Unkn","[Det","[Cond","[Punc","[Prop"]
    s=morphanalysis
    ilkpos="[Unkn"
    sonpos="[Unkn"
    while s[0]!="[":
        s=s[1:]
    if mod==0:
        for pos in Poslist:
            if pos in s[:5]:
                return pos[1:]
    if mod==1:
        a=0
        while a<len(s):
            s=s[a:]
            e=0
            for pos in Poslist:
                if s.find(pos)!=-1:
                    e=1
                    a=s.find(pos)+len(pos)
                    sonpos=pos
            if e==0:
                return sonpos[1:]
                break
        return sonpos[1:]
    return ilkpos[1:]
print(findPOS(s,1))
traincorp="Conllcorptrain.txt"
posfile=datawostags

İstanbul[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom] 9 
Prop


In [9]:
##take the POS tag information from the output of the disamb tool
## embed the tag into the training corpus
## assume we receive valid input
def addPOS(posdata,corpus):
    corpus=open(corpus).readlines()
    Poslucorp=open("sonposlu.traincorpus.txt","w")
    print(posdata[1].rsplit()[1])
    print(findPOS(posdata[1].rsplit()[1],1))
    #print(corpus[1]+posdata[1]+findPOS(posdata[1].rsplit()[1],0))
    sen=corpus[0].rsplit()
    for i in range(len(corpus)):
        sen=corpus[i].rsplit()
        Poslucorp.write(sen[0]+" ")
        Poslucorp.write(findPOS(posdata[i].rsplit()[1],1)+" ")
        for x in sen[1:]:
            Poslucorp.write(x+" ")
        Poslucorp.write("\n")
    
        

In [10]:
addPOS(posfile,traincorp)

şen[Adj]-lHk[Noun+Ness]+[A3sg]+SH[P3sg]+[Nom]
Noun


In [9]:
###Capitalization feature addition
import io
cap=io.open("ornekCapitalization.txt",encoding="utf-8").readlines()
for w in cap[0].rsplit():
    if w[0].isupper():
        print(w)
traincorp="Conllcorptrain.txt"
testcorp="Conllcorptest.txt"

##take input in Conll format and return for each word
#0 if not Capitalized
#1 if ProperName form e.g Arda
## 2 if allcaps e.g UPPERCASE
## 3 if mixedcase e.g. MixXED
def capExtractor(corpus,t):
    corp=io.open(corpus,encoding="utf-8").readlines()
    corp[0]=corp[0][1:]
    cinf=open("Capinfo"+t+".txt","w")
    for line2 in corp:
        line=line2.split()[0]
        if line[0].isupper():
            count=0
            count2=0
            for x in line[1:]:
                if x.isupper():
                    count=1
                else:
                    count2=1
            if count==0:
                cinf.write("1\n")
            if count2==0 and count==1:
                cinf.write("2\n") 
            if count2==1 and count==1:
                cinf.write("3\n")
        else:
            count=0
            for x in line[1:]:
                if x.isupper():
                    count=1
            if count==0:
                cinf.write("0\n")
            else:
                cinf.write("3\n")
#capExtractor(traincorp,"train")
capExtractor("Conllcorptest.txt","test")
 

Arda
Istanbul


In [12]:
##adds the cap feature to the corpus file
def capAdder(corpus,capfile,outname):
    corp=open(corpus).readlines()
    output=open(outname,"w")
    caps=open(capfile).read().split()
    for i in range(len(corp)):
        line =corp[i].split()
        cap=caps[i]
        for tok in line[:len(line)-1]:
            output.write(tok+" ")
        output.write(cap+" ")
        output.write(line[len(line)-1]+"\n")
capAdder("sonposlu.testcorpus.txt","Capinfotest.txt","caplisonposlutest.txt")  

In [15]:
##removes the s tags in a MPMD format text
def removestags(filename):
    data=open(filename).readlines()
    datawostags=[]
    tags=["<S>","</S>"]
    for line in data:
        if line.rsplit()[0] not in tags:
            datawostags.append(line)
    return datawostags

In [5]:
##extracts root for each word in the afterMPMD format
def rootExtractor(morpfile,outname):
    morpfwostags=removestags(morpfile)
    output=open(outname,"w")
    for line in morpfwostags:
        s=line.split()[1]
        root=""
        for i in range(s.find("[")):
            root+=s[i]
        output.write(root+"\n")
rootExtractor("afterMPMD.reyyan.test.txt","rootouttest.txt")

In [6]:
##add root features in the rootfile to the corpus
def rootAdder(corpus,rootfile,outname):
    corp=open(corpus).readlines()
    roots=open(rootfile).readlines()
    outfile=open(outname,"w")
    for i in range(len(corp)):
        corpline=corp[i].split()
        for x in corpline[:-1]:
            outfile.write(x+" ")
        outfile.write(roots[i].split()[0]+" ")
        outfile.write(corpline[-1]+"\n")
rootAdder("caplisonposlutest.txt","rootouttest.txt","rootlutest.txt")
        

In [7]:
##start of sentence adder
def ssAdder(corpus,mpmdfile,outname):
    corp=open(corpus).readlines()
    mpmd=open(mpmdfile).readlines()
    outf=open(outname,"w")
    mi=0
    for i in range(len(corp)):
        line=corp[i].split()
        if mpmd[mi].rsplit()[0]=="</S>":
            mi+=1
        if mpmd[mi].rsplit()[0]=="<S>":
            mi+=1
            for x in line[:-1]:
                outf.write(x+" ")
            outf.write("1 ")
            outf.write(line[-1]+"\n")
        else:
            for x in line[:-1]:
                outf.write(x+" ")
            outf.write("0 ")
            outf.write(line[-1]+"\n")            
        mi+=1

In [69]:
ssAdder("rootlutest.txt","afterMPMD.reyyan.test.txt","sslitest.txt")

<S>


In [8]:
##Prop extractor
def propextractor(morpfile,outname):
    morpfwostags=removestags(morpfile)
    output=open(outname,"w")
    for line in morpfwostags:
        s=line.split()[1]
        root=""
        if "[Prop]" in s:
            output.write("1\n")
        else:
            output.write("0\n")
propextractor("afterMPMD.reyyan.test.txt","proptest.txt")

In [12]:
props=open("proptrain.txt").readlines()
print(props[8])

1



In [74]:
def propadder(propfile,corpus,outname):
    corp=open(corpus).readlines()
    props=open(propfile).readlines()
    output=open(outname,"w")
    for x in range(len(corp)):
        line=corp[x].split()
        for l in line[:-1]:
            output.write(l+" ")
        output.write(props[x].split()[0]+" ")
        output.write(line[-1]+" \n")
#propadder("proptrain.txt","sslitrain.txt","propsstrain.txt")
#propadder("proptest.txt","sslitest.txt","propsstest.txt")

In [35]:
##changes the features values of the given index with the values in the featfilen file
def featurereplacer(corpus,featind,featfilen,outname):
    corp=open(corpus).readlines()
    feats=open(featfilen).readlines()
    out=open(outname,"w")
    for x in range(len(corp)):
        line=corp[x].split()
        featline=feats[x].split()
        for token in line[:featind]:
            out.write(token+" ")
        out.write(featline[0]+" ")
        for token2 in line[featind+1:]:
            out.write(token2+" ")
        out.write("\n")
featurereplacer("propsstest.txt",2,"Capinfotest.txt","newcappropsstest.txt")

In [36]:
##morpinfo of annotated words
def PropMorpInfo(morpfile,corpus):
    morpfwostags=removestags(morpfile) 
    corp=open(corpus).readlines()
    propmorp=[]
    for l in range(len(corp)):
        if corp[l].split()[-1]!="O":
            propmorp.append(morpfwostags[l].split()[1])
    return propmorp

In [12]:
propmorptrain=PropMorpInfo("afterMPMD.reyyan.train.txt","Conllcorptrain.txt")
propmorptest=PropMorpInfo("afterMPMD.reyyan.test.txt","Conllcorptest.txt")

In [13]:
count=0
for x in propmorptrain:
    if "[Acro]" in x:
        #print(x)
        count+=1
print(count)

3971


In [31]:
def morpinfoextract(morpfile,outfile):
    morpfwostags=removestags(morpfile) 
    out=open(outfile,"w")
    for x in morpfwostags:
        morp=x.split()[1]
        if "[Acro]" in morp:
            out.write("Acro\n")
            #print(morp)
        else:
            out.write("Notacro\n")

In [51]:
def morpinfonomextract(morpfile,outfile):
    morpfwostags=removestags(morpfile) 
    out=open(outfile,"w")
    for x in morpfwostags:
        morp=x.split()[1]
        w=x.split()[0]
        if "[Nom]" in morp and "-" not in morp and w[0].isupper():
            out.write("Nom\n")
        else:
            out.write("Notnom\n")

In [37]:
morpinfoextract("afterMPMD.reyyan.test.txt", "acrotest.txt")

In [39]:
propadder("acrotrain.txt","newcappropsstrain.txt","acrofinaltrain.txt")
propadder("acrotest.txt","newcappropsstest.txt","acrofinaltest.txt")

In [56]:
##investigation of the dataset
morptestfile=removestags("afterMPMD.reyyan.train.txt")
corp=open("Conllcorptrain.txt").readlines()
count=0
num=[str(x) for x in range(10)]
for x in range(len(corp)):
    if corp[x].split()[1]!="O":
        count+=1
        print(morptestfile[x].split()[1])
        #print(corp[x-1])
        print(corp[x])
        #print(corp[x+1])
print(count)

pozitif[Adj]
POZİTİF BORG

ve[Conj]
ve IORG

açık[Adj]
Açık IORG

radyo[Noun]+[A3sg]+[Pnon]+[Nom]
Radyo IORG

İstanbul[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İstanbul BLOC

askerî[Adj]+[NoHats]
Askeri BLOC

müze[Noun]+[A3sg]+[Pnon]+[Nom]
Müze ILOC

ve[Conj]
ve ILOC

kültür[Noun]+[A3sg]+[Pnon]+[Nom]
Kültür ILOC

site[Noun]+[A3sg]+SH[P3sg]+[Nom]
Sitesi ILOC

Anadolu[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Anadolu BLOC

Venedik[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
VENEDİK BLOC

Akira[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Akira BPER

Kurosawa[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Kurosawa IPER

Steven[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Steven BPER

Spielberg[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Spielberg IPER

Ryan[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Ryan BPER

Peter[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Peter BPER

Weir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Weir IPER

Truman[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Truman BPER

Stanley[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Stanley BPER

Kubrick[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Kubrick IPER

Tavia

Ergun BPER

Göknel[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Göknel IPER

Baykal[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Baykal BPER

Kars[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Kars BLOC

çevik[Adj]
Çevik BORG

kuvvet[Noun]+[A3sg]+[Pnon]+[Nom]
Kuvvet IORG

MHP[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
MHP BORG

CHP[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
CHP BORG

Akkaya[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Akkaya BLOC

Rusya[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
RUSYA BLOC

Yevgeni[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yevgeni BPER

Primakov[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Primakov IPER

Rusya[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Rusya BLOC

Yevgeni[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yevgeni BPER

Primakov[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Primakov IPER

Avrupa[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Avrupa BORG

birlik[Noun]+[A3sg]+SH[P3sg]+[Nom]
Birliği IORG

Avusturya[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Avusturya BLOC

Wolfgang[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Wolfgang BPER

Schuessel[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Schuessel IPER

Primakov[Noun]

Fener[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
FENER BORG

Rum[Noun]+[A3sg]+[Pnon]+[Nom]
Rum IORG

patrikhane[Noun]+[A3sg]+SH[P3sg]+[Nom]
Patrikhanesi IORG

Yunan[Noun]+[A3sg]+[Pnon]+[Nom]
Yunan BORG

Ortodoks[Adj]
Ortodoks IORG

kilise[Noun]+[A3sg]+SH[P3sg]+[Nom]
Kilisesi IORG

Brüksel[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Brüksel BLOC

Avrupa[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Avrupa BORG

birlik[Noun]+[A3sg]+SH[P3sg]+[Nom]
Birliği IORG

fener[Noun]+[A3sg]+[Pnon]+[Nom]
Fener BORG

Rum[Noun]+[A3sg]+[Pnon]+[Nom]
Rum IORG

patrikhane[Noun]+[A3sg]+SH[P3sg]+[Nom]
Patrikhanesi IORG

Türkiye[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Türkiye BLOC

ege[Noun]+[A3sg]+[Pnon]+[Nom]
Ege BORG

ordu[Noun]+[A3sg]+[Pnon]+[Nom]
Ordu IORG

komutanlık[Noun]+[A3sg]+SH[P3sg]+[Nom]
Komutanlığı IORG

Şirinyer[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Şirinyer BORG

asker[Noun]+[A3sg]+SH[P3sg]+[Nom]
Askeri IORG

cezaev[Noun]+[A3sg]+SH[P3sg]+[Nom]
Cezaevi IORG

Kayseri[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
KAYSERİ BLOC

kredi[Noun]+[A3sg]+[Pnon]+[


Monde[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Monde IORG

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

Türkiye[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Türkiye BLOC

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

CHP[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
CHP BORG

DP[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
DP BORG

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

Halit[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Halit BPER

ziya[Noun]+[A3sg]+[Pnon]+[Nom]
Ziya IPER

Uşaklıgil[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Uşaklıgil IPER

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

kordon[Noun]+[A3sg]+[Pnon]+[Nom]
Kordon BLOC

Narlıdere[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Narlıdere BLOC

Ege[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Ege BORG

palas[Noun]+[A3sg]+[Pnon]+[Nom]
Palas IORG

kültür[Noun]+[A3sg]+[Pnon]+[Nom]
Kültür BLOC

park[Noun]+[A3sg]+[Pnon]+[Nom]
Park ILOC

paraşüt[

Engin BPER

Berberoğlu[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Berberoğlu IPER

doğ[Verb]+[Pos]-YAn[Adj+PresPart]
Doğan BORG

holding[Noun]+[A3sg]+[Pnon]+[Nom]
Holding IORG

Aydın[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Aydın BPER

Doğan[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Doğan IPER

deniz[Noun]+[A3sg]+[Pnon]+[Nom]
Deniz BORG

kuvvet[Noun]+lAr[A3pl]+SH[P3sg]+[Nom]
Kuvvetleri IORG

Orhan[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Orhan BPER

karabulut[Noun]+[A3sg]+[Pnon]+[Nom]
Karabulut IPER

Kuşadası[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Kuşadası BLOC

Davut[Noun]+[Prop]+lAr[A3pl]+[Pnon]+[Nom]
Davutlar BLOC

değirmen[Noun]+[A3sg]+[Pnon]+[Nom]
Değirmen BORG

Restaurant[Unknown]
Restaurant IORG

DSP[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
DSP BORG

İzmir[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İzmir BLOC

Ahmet[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Ahmet BPER

Priştina[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Priştina IPER

Hakan[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Hakan BPER

tart[Verb]+[Pos]-YAn[Adj+PresPart]
Tartan IPER

Rifat[Noun]+[Prop]+[A3s

...


milliyet[Noun]+[A3sg]+[Pnon]+[Nom]
MİLLİYET BORG

AB[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
AB BORG

NATO[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
NATO BORG

Yugoslavya[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yugoslavya BLOC

BM[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
BM BORG

Kosova[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Kosova BLOC

NATO[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
NATO BORG

Kosova[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Kosova BLOC

ABD[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
ABD BLOC

Bill[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Bill BPER

Clinton[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Clinton IPER

Monica[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Monica BPER

Lewinsky[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Lewinsky IPER

Belçika[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
BELÇİKA BLOC

Semira[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Semira BPER

Adamu[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Adamu IPER

SPD[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
SPD BORG

BM[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
BM BORG

New[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
New BLOC

York[Noun]+[Prop]+[A3sg]+[Pnon]+[No

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

Ertaç[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Ertaç BPER

Tinar[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Tinar IPER

İsrail[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
İsrail BLOC

Türkiye[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Türkiye BLOC

Mehmet[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Mehmet BPER

Ağar[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Ağar IPER

Akşener[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Akşener BPER

meclis[Noun]+[A3sg]+[Pnon]+[Nom]
Meclis BORG

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

çakıcı[Noun]+[A3sg]+[Pnon]+[Nom]
Çakıcı BPER

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

Mesut[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Mesut BPER

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz IPER

Ecevit[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Ecevit BPER

DYP[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
DYP BORG

Celâl[Noun]+[NoHats]+[Prop]+[A3sg]+[Pnon]+[Nom]
Celal BPER

Ada[Noun]+[Prop]+[A3sg]+Hn[P2sg]+[Nom]
Adan IPER

Musa[Noun]+[Prop]+[

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

Susurluk[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Susurluk BLOC

Susurluk[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Susurluk BLOC

Susurluk[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Susurluk BLOC

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

Eyüp[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Eyüp BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık IPER

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

yeşil[Adj]
Yeşil BPER

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

Türkiye[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Türkiye BLOC

Bedük[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Bedük BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

meclis[Noun]+[A3sg]+[Pnon]+[Nom]
Meclis BORG

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Yılmaz BPER

çakıcı[Noun]+[A3sg]+[Pnon]+[Nom]


çakıcı[Noun]+[A3sg]+[Pnon]+[Nom]
Çakıcı BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

Eyüp[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Eyüp BPER

bey[Noun]+[A3sg]+[Pnon]+[Nom]
Bey IPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

çakıcı[Noun]+[A3sg]+[Pnon]+[Nom]
Çakıcı BPER

Eyüp[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Eyüp BPER

bey[Noun]+[A3sg]+[Pnon]+[Nom]
Bey IPER

çakıcı[Noun]+[A3sg]+[Pnon]+[Nom]
Çakıcı BPER

ABD[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
ABD BLOC

ABD[Noun]+[Acro]+[A3sg]+[Pnon]+[Nom]
ABD BLOC

çakıcı[Noun]+[A3sg]+[Pnon]+[Nom]
Çakıcı BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık BPER

Türkiye[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Türkiye BLOC

Eyüp[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Eyüp BPER

âşık[Noun]+[NoHats]+[A3sg]+[Pnon]+[Nom]
Aşık IPER

çakıcı[Noun]+[A3sg]+[Pnon]+[Nom]
Çakıcı BPER

Fransa[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Fransa BLOC

Yılmaz[Noun]+[Prop]+[A3sg]+[Pnon]+


Vrijberghe[Unknown]
Vrijberghe IPER

Coning[Unknown]
Coning IPER

Sharon[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Sharon BPER

Stone[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Stone IPER

Mike[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Mike BPER

Van[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
van IPER

Diem[Unknown]
Diem IPER

Hollanda[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Hollanda BLOC

Feyyaz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Feyyaz BPER

Tokar[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Tokar IPER

Hasan[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Hasan BPER

Pulur[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Pulur IPER

Feriye[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Feriye BORG

Karako[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]-lH[Adj+With]
Karakolu IORG

boğaz[Noun]+[A3sg]+[Pnon]+[Nom]
Boğaz BLOC

Abdülaziz[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Abdülaziz BPER

Hill[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Hill BLOC

si[Noun]+[A3sg]+[Pnon]+DA[Loc]
Side ILOC

bodrum[Noun]+[A3sg]+[Pnon]+[Nom]
Bodrum BLOC

Beyoğlu[Noun]+[Prop]+[A3sg]+[Pnon]+[Nom]
Beyoğlu BORG

sinema[Noun]+[A3sg]+SH[P3sg]+

In [52]:
morpinfonomextract("afterMPMD.reyyan.test.txt", "nomtest.txt")
morpinfonomextract("afterMPMD.reyyan.train.txt", "nomtrain.txt")

In [54]:
propadder("nomtrain.txt","acrofinaltrain.txt","nomacrofinaltrain.txt")
propadder("nomtest.txt","acrofinaltest.txt","nomacrofinaltest.txt")

In [1]:
def removeline(corpusname,outfile):
    corp=open(corpusname).readlines()
    out=open(outfile,'w')
    for line in corp:
        x=line.split()
        if x[0]!="2":
            out.write(line)

In [5]:
removeline("nomacrofinaltest.txt","2siznomacrotest.txt")

In [8]:
corp=open("2siznomacrotest.txt").readlines()

In [10]:
def prefixextract(corpusname):
    corp=open(corpusname).readlines()
    for line in corp:
        x=line.split()
        w=x[0]
        if len(w)>2:
            print(w)
            print w[-3:]

In [35]:
def findlasoccof(token,x):
    inv=""
    for i in token:
        inv=i+inv
    i=inv.find(x)
    ind=0
    if i==-1:
        ind=-1
    else:
        ind=len(token)-inv.find(x)-1   
    return ind

In [111]:
def inbrackets(token,ind):
    i1=findlasoccof(token[:ind],"[")
    i2=token[i1:].find("]")
    if i1<ind and ind<i2 and i1!=-1:
        return True
    else:
        return False

In [120]:
def lastinfsuffixextract(morpfile,outfile):
    datawostags=removestags(morpfile)
    out=open(outfile,"w")
    for line in datawostags:
        morp1=line.split()[1]
        str1=morp1
        ind=findlasoccof(str1,"+")
        ind2=str1[ind:].find("[")+ind
        while(ind2-ind<2 and ind!=-1):
            str1=str1[:ind-1]
            ind=findlasoccof(str1,"+")
            ind2=str1[ind:].find("[")+ind
        if ind==-1:
            out.write("None\n")
        else:
            #print(str1[ind+1:ind2])
            out.write(str1[ind+1:ind2]+"\n")

In [121]:
lastinfsuffixextract(morptest,"sufftest.txt")
lastinfsuffixextract(morptrain,"sufftrain.txt")

In [122]:
propadder("sufftrain.txt","nomacrofinaltrain.txt","sufnomtrain.txt")
propadder("sufftest.txt","nomacrofinaltest.txt","sufnomtest.txt")

In [2]:
removeline("sufres.txt","2sizsufres.txt")

In [ ]:
data=open("sufnomtrain.txt").readlines()
for line in data:
    